In [4]:
import pandas as pd
import numpy as np
import holidays
import main
import math
import pmdarima as pm

INPUT_PATH = "C:/Users/u00378/Desktop/tcc_est_unb"
#INPUT_PATH = "C:/Users/Igor/Desktop/TCC"

df = pd.read_pickle(f'{INPUT_PATH}/dados/tcc_data.pkl')

## Weekday and Holidays

In [5]:
br_holidays = holidays.country_holidays('BR')  # this is a dict
mg_holidays = holidays.country_holidays('BR', subdiv='MG')  # this is a dict

In [6]:
BR_H = list(map(lambda x: br_holidays.get(x), df['data_inversa']))
MG_H = list(map(lambda x: mg_holidays.get(x), df['data_inversa']))

WEEKDAYS = list(map(lambda x: x.weekday(), df['data_inversa']))

#df['event_name'] = map(br_holidays.get(), df['data_inversa'])

In [7]:
BR_H == MG_H

False

In [8]:
df['weekday'] = list(map(lambda x: x.weekday(), df['data_inversa']))
df = pd.get_dummies(df, columns=['weekday'], drop_first=True)

isnt_na = lambda x: False if bool(__import__('pandas').isna(x)) else True
filtro1 = np.array(list(map(isnt_na, df['uf'])))
filtro2 = np.array((list(map(isnt_na, df['br']))))

df_tcc = df[(filtro1) & (filtro2)]

## Ajuste

### Some tests

In [48]:
train = df_tcc[(df_tcc['uf'] == 'MG') & (df_tcc['data_inversa'] <= '2023-01-31')].groupby('data_inversa')['id'].agg('nunique').resample('D').sum()

test = df_tcc[(df_tcc['uf'] == 'MG') & (df_tcc['data_inversa'] >= '2023-02-01') & (df_tcc['data_inversa'] <= '2023-02-28')]\
    .groupby('data_inversa')['id'].agg('nunique').resample('D').sum()

In [49]:
WEEKDAYS = list(map(lambda x: x.weekday(), train.index))
MG_H = list(map(lambda x: 1 if mg_holidays.get(x) != None else 0, train.index))

exp_var1 = pd.DataFrame({'weekday': WEEKDAYS, 'is_holiday':MG_H})
exp_var1 = pd.get_dummies(exp_var1, columns=['weekday'], drop_first=True)


WEEKDAYS = list(map(lambda x: x.weekday(), test.index))
MG_H = list(map(lambda x: 1 if mg_holidays.get(x) != None else 0, test.index))

exp_var2 = pd.DataFrame({'weekday': WEEKDAYS, 'is_holiday':MG_H})
exp_var2 = pd.get_dummies(exp_var2, columns=['weekday'], drop_first=True)

In [106]:
from statsforecast import StatsForecast
Y_ts = pd.DataFrame({'ds':train.index, 'y':train.values})
Y_ts['unique_id'] = ['MG']*len(Y_ts)

StatsForecast.plot(Y_ts)

In [77]:
###############################################
### Serializing the ARIMA models (old form) ###
###############################################

#import joblib
#import pickle

#joblib.dump(model, 'arima.pkl')
#joblib_preds = joblib.load('arima.pkl').predict(test.shape[0], X = np.array(exp_var2))

In [120]:
#######################################
### Plotting the results (old form) ###
#######################################

#import matplotlib.pyplot as plt

#plt.style.use('fivethirtyeight')
#f = plt.figure()
#f.set_figwidth(10)
#f.set_figheight(10)

#plt.plot(train[-180:], c='blue',linewidth=2)
#plt.plot(test[-180:], c='red', linewidth=1.5)
#plt.plot(np.array(test.index), forecasts, c='green', linewidth=2)


#plt.show()

### Rolling Window